In [ ]:
# Bonus: To get rid of the grid as well:
#ax.xaxis.pane.set_edgecolor('#D0D0D0')
#ax.yaxis.pane.set_edgecolor('#D0D0D0')
#ax.zaxis.pane.set_edgecolor('#D0D0D0')
#ax.xaxis.pane.set_alpha(1)
#ax.yaxis.pane.set_alpha(1)
#ax.zaxis.pane.set_alpha(1)
#ax.set_yticks([72,74,76,78])
#ax.set_xticks([-10,0,10])
# changing grid lines thickness of Y axis to 1 and giving color to red
#ax.set_xlabel("Longitude / deg",linespacing=10)
#ax.set_ylabel("Latitude / deg N",linespacing=6.5)
# To use a custom hillshading mode, override the built-in shading and pass
# in the rgb colors of the shaded surface calculated from "shade".
#rgb = ls.shade(z, cmap=cm.gist_earth, vert_exag=0.1, blend_mode='soft')

def map_AR_sea_ice(cfg_dict,radar_ds,ds):
    import matplotlib
    import cartopy
    import cartopy.crs as ccrs
    import cartopy.io.img_tiles as cimgt
    
    sector_colors={"warm_sector":"orange",
                  "cold_sector":"purple",
                  "internal":"grey"}
    
    x, y, z = radar_ds["lon"], radar_ds["lat"], radar_ds["alt"]
    
    matplotlib.rcParams.update({"font.size":16})
    orig_map = plt.cm.get_cmap('Blues') # getting the original colormap using cm.get_cmap() function
    reversed_map = orig_map.reversed()  # reversing the original colormap using reversed() function
                                            # normally the actual bahamas file is used from HALO-(AC)3. However,
                                            # this is not feasible now for testing

    add_quiver=True
    sea_ice_file=cfg_dict["device_data_path"]+"sea_ice/"+\
                        "asi-AMSR2-n6250-"+cfg_dict["date"]+"-v5.4.nc"

    seaice = xr.open_dataset(sea_ice_file)
    seaice = seaice.seaice
    # Create a Stamen terrain background instance.
    stamen_terrain = cimgt.Stamen('terrain-background')

    with_sondes=True    
    llcrnlat = 68
    llcrnlon = -10
    urcrnlat = 82
    urcrnlon = 20

    extent = [llcrnlon-5, urcrnlon+5, llcrnlat, urcrnlat]
    coordinates=dict(EDMO=(11.28, 48.08), 
                         Keflavik=(-22.6307, 63.976),
                         Kiruna=(20.336, 67.821),
                         Bergen=(5.218, 60.293),
                         Longyearbyen=(15.46, 78.25),
                         Lerwick=(-1.18, 60.13),
                         Ittoqqortoormiit=(-21.95, 70.48),
                         Tasiilaq=(-37.63, 65.60))
    # set plotting options
    plt.rcdefaults()
    # get plot properties
    props = dict(Flight_20210705b=dict(figsize=(4, 4),
                        cb_loc="bottom", shrink=1, l_loc=1))
    matplotlib.rcParams.update({"font.size":12})
    # start plotting
    fig, ax = plt.subplots(figsize=(10,8), subplot_kw={"projection": ccrs.NorthPolarStereo()})
    ax.add_image(stamen_terrain, 4)
    ax.coastlines(resolution="50m")
    ax.add_feature(cartopy.feature.BORDERS)
    ax.set_extent(extent, crs=ccrs.PlateCarree())
    gl = ax.gridlines(crs=ccrs.PlateCarree(), draw_labels=True, 
                          x_inline=False, y_inline=False)
    gl.bottom_labels = False
    gl.right_labels  = False
    # add sea ice extent
    ax.pcolormesh(seaice.lon, seaice.lat,seaice, 
                          transform=ccrs.PlateCarree(), cmap=reversed_map,
                          alpha=0.9)

    x1,y1 = coordinates["Kiruna"]   
    x2, y2 =coordinates["Longyearbyen"]
    ax.plot(x1, y1, '.r', markersize=15, markeredgecolor="k",
            transform=ccrs.PlateCarree(),zorder=10)

    ax.plot(x2, y2, '.r', markersize=15, markeredgecolor="k",
            transform=ccrs.PlateCarree(),zorder=10)

    ax.text(x2 + 2, y2 - 0.35, "Longyearbyen\n(LYR)", fontsize=9,
             transform=ccrs.PlateCarree(),color="red",
             bbox=dict(facecolor='lightgrey',edgecolor="black"),zorder=12)
    ax.text(x1 - 2, y1 + 0.9, "Kiruna (KRN)", fontsize=9,
             transform=ccrs.PlateCarree(),color="red",
             bbox=dict(facecolor='lightgrey',edgecolor="black"),zorder=12)

    ax.plot(radar_ds["lon"],radar_ds["lat"],z.mean()/1000,color="grey",linewidth=2,ls="--",zorder=4,
           transform=ccrs.PlateCarree())
    ax.plot(x, y, z.mean()/1000,linewidth=4,color="grey",zorder=4,
           transform=ccrs.PlateCarree())


    ## Show ERA5-AR
    #if show_AR_detection:    
    import atmospheric_rivers as AR
    AR=AR.Atmospheric_Rivers("ERA",use_era5=True)
    flight_date="20220315"
    AR_era_ds=AR.open_AR_catalogue(after_2019=int(flight_date[0:4])>2019,
            year=cmpgn_cls.year,month=cmpgn_cls.flight_month[flight[0]])
    AR_era_data=AR.specify_AR_data(AR_era_ds,flight_date)
    i=12 # hour time step to consider
    hatches=plt.contourf(AR_era_ds.lon,AR_era_ds.lat,
                AR_era_ds.shape[0,AR_era_data["model_runs"].start+i,0,:,:],
                hatches=["//"],cmap="bone_r",alpha=0.2,
                transform=ccrs.PlateCarree())
    for c,collection in enumerate(hatches.collections):
        collection.set_edgecolor("k")

    ##### Quivers for IVT magnitude and direction
    #-----------------------------------------------------------------#
    # Quiver-Plot
    if add_quiver:
        step=15
        quiver_lon=np.array(ds["longitude"][::step])
        quiver_lat=np.array(ds["latitude"][::step])
        u=ds["IVT_u"][i,::step,::step]
        v=ds["IVT_v"][i,::step,::step]
        v=v.where(v>200)
        v=np.array(v)
        u=np.array(u)
        quiver=plt.quiver(quiver_lon,quiver_lat,
                    u,v,color="lightgreen",edgecolor="k",lw=1,
                                      scale=800,scale_units="inches",
                                      pivot="mid",width=0.008,
                                      transform=ccrs.PlateCarree())
        plt.rcParams.update({'hatch.color': 'lightgreen'})
    #-----------------------------------------------------------------#            

    for sector in ["warm_sector","cold_sector","internal"]:
        if not sector=="internal":
            for in_out in ["in","out"]:
                sonde_no=relevant_sondes_dict[sector][in_out]
                ax.scatter(Dropsondes["Lon"].iloc[sonde_no],
                        Dropsondes["Lat"].iloc[sonde_no],
                        color=sector_colors[sector],marker="v",
                          s=100,edgecolor="k",transform=ccrs.PlateCarree(),lw=2,zorder=10)
        else:
            sonde_no=relevant_sondes_dict[sector],
            ax.scatter(Dropsondes["Lon"].iloc[sonde_no],
                    Dropsondes["Lat"].iloc[sonde_no],
                    color=sector_colors[sector],marker="v",
                    s=100,edgecolor="k",lw=2,zorder=10,transform=ccrs.PlateCarree())



    y_pos=0.85
    sonde_shapes=["v","s"]
    fig_name=flight[0]+"_AR_IWV_IVT_Track_Map_"
    if with_sondes:
        fig_name+="with_sondes"
    fig_name+=".png"
    fig.savefig(plot_path+fig_name,bbox_inches="tight",dpi=600)
    print("Figure saved as:",plot_path+fig_name)

In [ ]:
def map_3d_radar_view(processed_radar,Dropsondes,plot_path):
    import numpy as np
    import itertools
    import cartopy.feature
    from cartopy.mpl.patch import geos_to_path

    matplotlib.rcParams.update({"font.size":22})
    take_all=False
    if take_all:
        curved_radar=processed_radar.copy()#sel({"time":slice(time_start,time_end)})
    else:
        time_start=pd.Timestamp(internal_times[0])-pd.Timedelta("15min")
        time_end  =pd.Timestamp(internal_times[0])+pd.Timedelta("15min")
        curved_radar=processed_radar.sel({"time":slice(time_start,time_end)})

    # Radar period
    curved_radar=pd.DataFrame(data=np.array(processed_radar["dBZg"][:]),
                              index=pd.DatetimeIndex(np.array(processed_radar["dBZg"].time[:])),
                              columns=np.array(processed_radar["height"][:]))

    curved_radar = curved_radar[~curved_radar.index.duplicated(keep='first')]
    # This import registers the 3D projection, but is otherwise unused.
    x, y, z = processed_radar["lon"], processed_radar["lat"], processed_radar["alt"]
    x_contour=np.tile(x,(len(processed_radar["height"]),1)).T
    y_contour=np.tile(y,(len(processed_radar["height"]),1)).T

    z_contour_1d=np.array(processed_radar["height"][:])
    z_contour=np.tile(z_contour_1d,(len(processed_radar["time"]),1))
    # Set up plot
    inflow_st_ind=curved_radar.index.get_loc(inflow_times[0]).start
    inflow_end_ind=curved_radar.index.get_loc(inflow_times[-1]).start
    internal_st_ind=curved_radar.index.get_loc(internal_times[0]).start
    internal_end_ind=curved_radar.index.get_loc(internal_times[-1]).start
    outflow_st_ind=curved_radar.index.get_loc(outflow_times[0]).start
    outflow_end_ind=curved_radar.index.get_loc(outflow_times[-1]).start


    fig, ax = plt.subplots(subplot_kw=dict(projection='3d'),figsize=(22,12))
    # Plot dropsonde profiles
    sector_colors={"warm_sector":"orange",
                  "cold_sector":"purple",
                  "internal":"grey"}
    ax.scatter(x_contour[inflow_st_ind:inflow_end_ind,:].flatten(),y_contour[inflow_st_ind:inflow_end_ind,:].flatten(),
               z_contour[inflow_st_ind:inflow_end_ind,:].flatten()/1000,s=0.1,
              c=np.array(curved_radar.iloc[inflow_st_ind:inflow_end_ind,:].values[:]),cmap=cmaeri.roma_r,vmin=-30,vmax=30,
              zorder=2)
    ax.scatter(x_contour[internal_st_ind:internal_end_ind,:].flatten(),y_contour[internal_st_ind:internal_end_ind,:].flatten(),
               z_contour[internal_st_ind:internal_end_ind,:].flatten()/1000,s=0.1,
              c=np.array(curved_radar.iloc[internal_st_ind:internal_end_ind,:].values[:]),cmap=cmaeri.roma_r,vmin=-30,vmax=30,
               zorder=1)
    # Often Unused for optic reasons
    #ax.scatter(x_contour[outflow_st_ind:outflow_end_ind,:].flatten(),y_contour[outflow_st_ind:outflow_end_ind,:].flatten(),
    #       z_contour[outflow_st_ind:outflow_end_ind,:].flatten()/1000,s=0.1,
    #       c=np.array(curved_radar.iloc[outflow_st_ind:outflow_end_ind,:].values[:]),cmap=cmaeri.roma_r,vmin=-30,vmax=30,zorder=0)

    ax.plot(radar_ds["lon"],radar_ds["lat"],z.mean()/1000,color="grey",linewidth=2,ls="--",zorder=4)
    ax.plot(x, y, z.mean()/1000,linewidth=4,color="grey",zorder=4)

    for sector in ["warm_sector","cold_sector","internal"]:
            if not sector=="internal":
                for in_out in ["in","out"]:
                    sonde_no=relevant_sondes_dict[sector][in_out]
                    relevant_Sondes=Dropsondes["Lon"].iloc[sonde_no]

                    ax.scatter(Dropsondes["Lon"].iloc[sonde_no],
                            Dropsondes["Lat"].iloc[sonde_no],
                            z.mean()/1000,
                            color=sector_colors[sector],marker="v",
                              s=250,edgecolor="k",lw=3,zorder=10)
                    try:
                        ax.plot([Dropsondes["Lon"].iloc[sonde_no],Dropsondes["Lon"].iloc[sonde_no]],
                           [Dropsondes["Lat"].iloc[sonde_no],Dropsondes["Lat"].iloc[sonde_no]],
                           [0,z.mean()],ls="--",color="k",lw=3,zorder=10) #--> the problem is the z vector for length 2
                    except:
                        pass

            else:
                sonde_no=relevant_sondes_dict[sector],
                ax.scatter(Dropsondes["Lon"].iloc[sonde_no],
                        Dropsondes["Lat"].iloc[sonde_no],
                        z.mean()/1000,
                        color=sector_colors[sector],marker="v",
                        s=250,edgecolor="k",lw=3,zorder=10)
                try:
                    ax.plot([Dropsondes["Lon"].iloc[sonde_no],Dropsondes["Lon"].iloc[sonde_no]],
                           [Dropsondes["Lat"].iloc[sonde_no],Dropsondes["Lat"].iloc[sonde_no]],
                           [0,z.mean()/1000],ls="--",color="k",lw=3,zorder=10) 
                except:
                    pass

    #    # Set view angle
    ax.xaxis._axinfo['juggled'] = (1,2,0) #---> to be uncommented
    ax.view_init(40,290)
    ax.set_zlim([0,12])
    ax.set_xlim([processed_radar["lon"].values.min(),
                 processed_radar["lon"].values.max()])
    ax.set_ylim([processed_radar["lat"].values.min(),processed_radar["lat"].values.max()])
    # changing grid lines thickness of x axis to 1
    # Get rid of colored axes planes
    # First remove fill#
    ax.xaxis.pane.fill = False
    ax.yaxis.pane.fill = False
    ax.zaxis.pane.fill = False
    # Now set color to white (or whatever is "invisible")
    ax.xaxis.pane.set_edgecolor('w')
    ax.yaxis.pane.set_edgecolor('w')
    ax.zaxis.pane.set_edgecolor('w')

    ax.xaxis._axinfo["grid"].update({"linewidth":1,"color":"w"})
    ax.yaxis._axinfo["grid"].update({"linewidth":2,"color":"w"})
    ax.zaxis._axinfo["grid"].update({"linewidth":2,"color":"w"})
    ax.xaxis.set_tick_params(width=10)
    ax.yaxis.set_tick_params(width=0)
    ax.zaxis.set_tick_params(width=10)

    ax.set_zlabel("Height (km)",linespacing=3.1)

    sns.despine(offset=10,ax=ax)
    fig_name="RF05_Budget_Area.png"
    fig.savefig(plot_path+fig_name,dpi=300,bbox_inches="tight")
    print("Figure saved as:",plot_path+fig_name)